#property indicator_chart_window
#property indicator_buffers 1
#property indicator_plots   1

#property indicator_label1  "Tick Developing POC"
#property indicator_type1   DRAW_LINE
#property indicator_color1  clrDeepSkyBlue
#property indicator_width1  2

// ─────────────────────────────────────
input double PriceStepMult = 1.0;

double PocLine[];

// --- internal state
double priceLow   = 0.0;
double priceHigh  = 0.0;
double step;
bool   initialized = false;

double volMap[];
int    volSize = 0;
double currentPOC = EMPTY_VALUE;

// ─────────────────────────────────────
int OnInit()
{
   SetIndexBuffer(0, PocLine, INDICATOR_DATA);
   ArraySetAsSeries(PocLine, true);

   step = _Point * PriceStepMult;
   return INIT_SUCCEEDED;
}

// ─────────────────────────────────────
void OnTick()
{
   double bid = SymbolInfoDouble(_Symbol, SYMBOL_BID);

   if(!initialized)
   {
      priceLow  = bid;
      priceHigh = bid;
      volSize   = 1;
      ArrayResize(volMap, 1);
      volMap[0] = 1.0;
      initialized = true;
      currentPOC = bid;
      return;
   }

   if(bid < priceLow)
   {
      int add = int((priceLow - bid) / step) + 1;
      int oldSize = volSize;
      volSize += add;

      ArrayResize(volMap, volSize);
      for(int i = oldSize; i < volSize; i++)
         volMap[i] = 0.0;

      priceLow -= add * step;
   }
   else if(bid > priceHigh)
   {
      int add = int((bid - priceHigh) / step) + 1;
      int oldSize = volSize;
      volSize += add;

      ArrayResize(volMap, volSize);
      for(int i = oldSize; i < volSize; i++)
         volMap[i] = 0.0;

      priceHigh += add * step;
   }

   int idx = int((bid - priceLow) / step);
   if(idx >= 0 && idx < volSize)
      volMap[idx] += 1.0;   // one tick

   // --- recompute POC
   double maxVol = 0.0;
   int pocIdx = -1;

   for(int i = 0; i < volSize; i++)
   {
      if(volMap[i] > maxVol)
      {
         maxVol = volMap[i];
         pocIdx = i;
      }
   }

   if(pocIdx >= 0)
      currentPOC = priceLow + pocIdx * step;
}

// ─────────────────────────────────────
int OnCalculate(
   const int rates_total,
   const int prev_calculated,
   const datetime& time[],
   const double& open[],
   const double& high[],
   const double& low[],
   const double& close[],
   const long& tick_volume[],
   const long& volume[],
   const int& spread[]
)
{
   if(!initialized)
      return 0;

   int start = (prev_calculated > 0) ? prev_calculated - 1 : 0;

   for(int i = start; i < rates_total; i++)
      PocLine[i] = currentPOC;

   return rates_total;
}
